### Permissioning non-standardized data in the ADE
Technically speaking, access to a table in the Agency Data Exchange is defined by:
- the Access Control Entry (ACE): the identifier of the user (Step 3 below)
- the Access Control List Key (ACL-key): the identifier of the data to give access on (Steps 1 & 2 below)

As in the previous notebooks, we start by loading the libraries, setting up OpenLattice authentication, and loading OpenLattice API-clients.  
_Note_ for instructions on how to install the `openlattice` python library of APIs, please see [this link](https://help.openlattice.com/article/104-overview-and-installation-of-api-clients)

In [7]:
import pandas as pd
import openlattice
import requests
import olpy
import uuid
import sys
import os

Set up your Openlattice authentication. Find your jwt token from the tab called `Account` in the upper left after logging in here: https://
openlattice.com/orgs/#/orgs.


In [8]:
baseurl = 'https://api.openlattice.com' # prod
jwt = "***"

configuration = openlattice.Configuration()
configuration.host = baseurl
configuration.access_token = jwt

Import the necessary Openlattice APIs for the tasks at hand. These rely on the "configuration" object we made above, that holds one's authentication.

In [9]:
principalAPI = openlattice.PrincipalApi(openlattice.ApiClient(configuration))
edmAPI = openlattice.EdmApi(openlattice.ApiClient(configuration))
entitysetsAPI = openlattice.EntitySetsApi(openlattice.ApiClient(configuration))
permissionsAPI = openlattice.PermissionsApi(openlattice.ApiClient(configuration))
dataAPI = openlattice.DataApi(openlattice.ApiClient(configuration))
datasetAPI = openlattice.DatasetApi(openlattice.ApiClient(configuration))
orgAPI = openlattice.OrganizationsApi(openlattice.ApiClient(configuration))
authorizationsAPI = openlattice.AuthorizationsApi(openlattice.ApiClient(configuration))

### 1. Creating the ACL-key:  Look up the id_table and id_column
In the OpenLattice backend, each table and column within it has a unique identifying UUID (in case names might overlap), that one needs to input while using the OpenLattice Permissions API.  
If one is a member of an organization and knows the OrgID, you can grab the datasets and their metadata for all data that you have access to.
- obtain the Organization ID from the Openlattice Orgs app, in the landing page for that specific organization. (https://openlattice.com/orgs/)

In [10]:
OrgId = '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8' #Openlattice Demo Org

In [11]:
atlas_tables = datasetAPI.get_external_database_tables(
    organization_id = OrgId)
print(len(atlas_tables))
atlas_tables

3


[{'description': '',
  'id': '25193301-2413-4054-89e6-ef9721f5cec6',
  'name': 'zzz_bhr_entities_clean_2020_5_26_17_27_3_PST',
  'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
  'title': 'zzz_bhr_entities_clean_2020_5_26_17_27_3_PST'},
 {'description': '',
  'id': '3c0262e1-075a-4d1a-a73d-440ba619929e',
  'name': 'fake_people',
  'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
  'title': 'fake_people'},
 {'description': 'This is a dataset including demo arrests',
  'id': 'f92f80d7-1288-4cd2-9e1c-2d6393b03b94',
  'name': 'demo_arrests',
  'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
  'title': 'demo_arrests'}]

Many times one has to filter out only the tables we want to permission at the moment. Here I've used a partial string search to grab only the tables I want in this example. 

In [14]:
subset_tables = list(filter(lambda x: "bhr" not in x.name, atlas_tables))
print(len(subset_tables))
subset_tables

2


[{'description': '',
  'id': '3c0262e1-075a-4d1a-a73d-440ba619929e',
  'name': 'fake_people',
  'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
  'title': 'fake_people'},
 {'description': 'This is a dataset including demo arrests',
  'id': 'f92f80d7-1288-4cd2-9e1c-2d6393b03b94',
  'name': 'demo_arrests',
  'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
  'title': 'demo_arrests'}]

There's often too many atlas tables (and with a lot of columns each) to grab all table and column IDs at once. We will therefore put it in the permissioning loop instead.  
  
  If you're interested in inspecting the objects in the loop further down, this is some code that I was using to figure things out.

In [ ]:
### Testing out code for the loop

# subset_tables[0].id                 #table ID 
# [x.id for x in subset_tables]         #all tableIDs in a list


## Dictionary with keys 'columns', 'table'. List of dicts, one for each column in the table
# table1_info = datasetAPI.get_external_database_table_with_column_metadata(
#     organization_id = OrgId, table_id = subset_tables[0].id)     

# table1_info.columns          
# test = [x.id for x in table1_info.columns]           # list of columns IDs
# test[0:2]

### 2. Create the principal object that one is giving access to.

If we want to add permissions to a new user, we can search for their unique user id with the `principalApi`.  For example, if I want to look up myself (Kim Engie), I can search for my last name (less common than my first). In this example we are looking up a headless account, `datascience@openlattice.com`.  

What we need is the `user_id` for each person. To give permissions to > 1 user at a time, find all of their principals and put them in a list.  
  
_TIPS:_ If you can't find someone by last name try putting a `*` in front of it (e.g., `*engie`)

**Permissioning to individual users**

In [ ]:
## find principals for users 
principalAPI.search_all_users("kim")

In [13]:
## Create the principal. 

# for one person
# perm_principal = openlattice.Principal(type="USER", id= "google-oauth2|106178747462282285391") 

# for several people
# users
users = [
    'auth0|5efbb1d8228149001965962a', # Yue
    'google-oauth2|104846254649175859508', # Kim
    'google-oauth2|105940757369523086021' # Nate
]

#### Permissioning to roles

If we want to add permissions to a role, we can search for its id with the `orgApi`.  Find the OrgID in the Openlattice orgs app here: https://
openlattice.com/orgs/#/orgs.  
  
To give permissions to > 1 role at a time, find all of their principals and put them in a list.  

In [15]:
# This code is to pull up the roles you want, to be used in the next cell.
OrgId = '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8' # Openlattice Demo Org
org = orgAPI.get_organization(OrgId)
org

{'apps': ['53b897aa-e3ac-4952-bf85-3a5822882c78',
          'c787c494-61f5-43c7-8831-8c7a11c2740a',
          'a77cd2d1-03c4-4dae-ad5c-a836251310fd',
          '26abca5c-4502-4d8c-80ce-795807266881',
          '570488a2-5ea2-44eb-92bc-07e88c6440f3',
          '73dd2cc3-38ec-405f-b551-c8c69b45bdcf',
          'a3ba1424-71d9-4fb2-8671-6e94f4e0b7c3',
          'fa6452b9-bd57-48af-a217-b289d453b4fb',
          '77cb2821-38ab-4ffb-b923-8965781f0697',
          '169a87a9-f88c-4bb8-93b8-6025dca19b0e',
          '100d99db-b163-4b4c-a9bd-d3c80b57d34c',
          '7d2f9be9-8168-433a-921f-99422248f279',
          '1a0f6f10-bbc8-47ee-b0cd-086a52825b43'],
 'description': '*Organization for Demo purposes. Data belonging to this org '
                'are fictional and used for demonstration purposes.*',
 'emails': None,
 'id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
 'members': [{'id': 'auth0|5e3984d7acc5260ee96d483e', 'type': 'USER'},
             {'id': 'google-oauth2|101142855019542997876', 'type

In [9]:
### Create the principal for roles

## For one role - 
# roleid = "1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8|DemoData_Write"

## For several
recipient_perms = [
        ("1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8|DemoData_Read", ["READ"]),
        ("1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8|DemoData_Write", ["WRITE"]),
        ("1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8|DemoData_Owner", ["OWNER"])
    ]

### 3. Give permissions
We loop over the dictionary constructed in Step 1, and grant access to the principal in Step 2. Again, access is defined by:
- the Access Control Entry (ACE): the identifier of the user
- the Access Control List Key (ACL-key): the identifier of the data to give access on

The permissions API goes through and gives permissions to each column in a table separately.  
  
NOTE: for column permissions, one needs to specify both the tableID and columnID

**Example 1.** Using `update_acls`, one call per table for ALL permissions we will give to this table. We are adding permissions for 3 roles, to all columns, and to the table object itself as well, in 1 call.

In [ ]:
# For multiple roles
for table in subset_tables:
    acl_data = []
    for x in recipient_perms:
        perm_principal = openlattice.Principal(type="ROLE", id= x[0]) 
        print(f"Gathering ACLs for {x[1]} access to columns in {table.name}")    

        ace = openlattice.Ace(
                principal = perm_principal,
                permissions = x[1]
            )
        # get column IDs
        table_info = datasetAPI.get_external_database_table_with_column_metadata(
            organization_id = OrgId, table_id = table.id)
        columnIDs = [x.id for x in table_info.columns]
        
        # Get acl for each column
        for column in columnIDs:
            col_acldata = openlattice.AclData(action = "ADD",
                                              acl = openlattice.Acl(acl_key = [table.id, column],
                                                                    aces = [ace]))

            acl_data.append(col_acldata)

    # Get acl for the table itself
    t_acldata = openlattice.AclData(action = "ADD",
                                  acl = openlattice.Acl(acl_key = [table.id],aces = [ace]))
    print(f"Gathered all ACLs for access to table {table.name}")

    acl_data.append(t_acldata)
#     print(len(acl_data))
                                     
    permissionsAPI.update_acls(acl_data)
    print(f"Updated permissions to table {table.name}")


**Example 2.** Assigning permissions to multiple users at a time. This is using the original `update_acl` call, which only contains 1 permission change per request - its harder on the stack. Hasn't been modified yet to use `update_acls` but can to be similar to Example 1.

In [ ]:
# for multiple people
for user in users:
    perm_principal = openlattice.Principal(type="USER", id= user) 
    print(f"Giving access for {user}")
    for table, columns in tables_to_give_access.items():
        for column in columns:
            print("Giving access on column %s on table %s to %s"%(column, table, user))
    #         row = datasets_with_columns.loc[(table, column)]
            row = datasets_with_columns.loc[(datasets_with_columns['name_table'] == table) & \
                                            (datasets_with_columns['name_column'] == column)]
            ace = openlattice.Ace(
                principal = perm_principal,
                permissions = ["READ", "WRITE", "OWNER"]
            )

            acldata = openlattice.AclData(action = "ADD",
                                acl = openlattice.Acl(acl_key = [str(row.id_table.iloc[0]), str(row.id_column.iloc[0])],aces = [ace])
                                         )
            permissionsAPI.update_acl(acldata)

**Example 3.** Same as example 2, but if you are only gicving permissions to one user.

In [38]:
# for 1 person
for table, columns in tables_to_give_access.items():
    for column in columns:
        print("Giving access on column %s on table %s"%(column, table))
#         row = datasets_with_columns.loc[(table, column)]
        row = datasets_with_columns.loc[(datasets_with_columns['name_table'] == table) & \
                                        (datasets_with_columns['name_column'] == column)]
        ace = openlattice.Ace(
            principal = perm_principal,
            permissions = ["READ"]
        )

        acldata = openlattice.AclData(action = "ADD",
                            acl = openlattice.Acl(acl_key = [str(row.id_table.iloc[0]), str(row.id_column.iloc[0])],aces = [ace])
                                     )
        permissionsAPI.update_acl(acldata)

Giving access on column Name on table cdss_facilities_from_agol
Giving access on column Program on table cdss_facilities_from_agol
Giving access on column Facility_Number on table cdss_facilities_from_agol
Giving access on column Facility_Type on table cdss_facilities_from_agol
Giving access on column Serves on table cdss_facilities_from_agol
Giving access on column Serves_Code on table cdss_facilities_from_agol
Giving access on column Capacity on table cdss_facilities_from_agol
Giving access on column Status on table cdss_facilities_from_agol
Giving access on column Telephone on table cdss_facilities_from_agol
Giving access on column Address on table cdss_facilities_from_agol
Giving access on column City on table cdss_facilities_from_agol
Giving access on column Zip_Code on table cdss_facilities_from_agol
Giving access on column Latitude on table cdss_facilities_from_agol
Giving access on column Longitude on table cdss_facilities_from_agol
Giving access on column FID on table cdss_fac